In [89]:
import random


class OrderBook:
    
    def __init__(self):
        
        self.bids = {} # bid prices and quantities
        self.asks = {} # ask prices and quantities
             
    def add_bid(self, price, size):
        self.bids[price] = size
        
    def add_ask(self, price, size):
        self.asks[price] = size
        
    def update_bid(self, price, delta):
        self.bids[price] += delta
        
    def update_ask(self, price, delta):
        self.asks[price] += delta
        
    def delete_bid(self, price):
        del self.bids[price]
        
    def delete_ask(self, price):
        del self.asks[price]
        
    def best_bid(self):
        best_bid = min(self.bids, key=self.bids.get)
        best_bid_size = self.bids[best_bid]
        return best_bid_size, best_bid

    def best_ask(self):
        best_ask = max(self.asks, key=self.asks.get)
        best_ask_size = self.asks[best_ask]
        return best_ask, best_ask_size
    
    def spread(self):
        return self.best_ask()[0] - self.best_bid()[1]
    
    def market_order(self):
        
        if self.spread() > 0:
            return [*self.best_bid(), *self.best_ask()]
        
        
        
        


In [100]:
import numpy as np
from scipy.stats import poisson

# Parameters of the Hawkes process
mu = 0.1 # expected rate of market orders
alpha = 0.2 # self-excitement parameter
beta = 0.1 # mutual-excitement parameter

# Number of market orders to generate
n = 100

# Initialize the times of market orders
market_times = [0]

# Generate market order times using the Hawkes process
for i in range(1, n):
    # Compute the conditional intensity at time t
    lambda_t = mu + sum(alpha * np.exp(-beta * (market_times[i-1] - market_times[j])) for j in range(i))
    # Generate the next market order time using a Poisson process with intensity lambda_t
    market_times.append(market_times[i-1] + poisson.rvs(lambda_t))

    
# Initialize the order book
ob = OrderBook()

mo = []

# Add some bids and asks to the order book
for timestamp in market_times:
    # Add a bid at the timestamp with a random size
    ob.add_bid(timestamp, np.random.randint(1, 25))
    ob.add_ask(timestamp, np.random.randint(1, 25))
    if ob.market_order() is not None:
        
        mo.append((timestamp, 
                   ob.market_order()[0],
                   ob.market_order()[1],
                   ob.market_order()[2],
                   ob.market_order()[3])
                 )
        
    
# Get the current best bid and ask prices and sizes

# Create a DataFrame to store the best bid and ask prices
prices = pd.DataFrame(mo)

prices.columns = ['timestamp', 'bid_size', 'bid', 'ask', 'ask_size']


prices

                            

,timestamp,bid_size,bid,ask,ask_size
0,34,1,13,34,24
1,42,1,13,42,24
2,44,1,13,42,24
3,48,1,13,42,24
4,49,1,13,42,24
...,...,...,...,...,...
66,144,1,13,42,24
67,146,1,13,42,24
68,146,1,13,42,24
69,148,1,13,42,24


In [ ]:
class HawkesProcess:
    def __init__(self, mu, a, b, intensity_func=None, intensity_dict=None):
        self.mu = mu
        self.a = a
        self.b = b
        self.intensity_func = intensity_func
        self.intensity_dict = intensity_dict
        if intensity_dict:
            self.intensity_func = intensity_dict.get(intensity_func)

    def hawkes_rhs(self, intensity, t, events):
        past_intensity = np.sum(self.intensity_func(t, events, self.a, self.b))
        return self.mu + past_intensity

    def log_likelihood(self, params, data):
        mu, a, b = params
        self.mu = mu
        self.a = a
        self.b = b
        events = data
        intensity = np.zeros(len(events))
        for i, t in enumerate(events[:-1]):
            intensity[i+1] = odeint(self.hawkes_rhs, intensity[i], [t, events[i+1]], args=(events[:i+1],))[-1][0]
        log_lik = -np.sum(intensity) - np.sum(np.log(intensity))
        return log_lik

    def fit(self, data, method='L-BFGS-B', bounds=None):
        from scipy.optimize import minimize
        self.events = data
        params_init = [self.mu, self.a, self.b]
        if bounds is None:
            bounds = [(1e-5, None), (1e-5, None), (1e-5, None)]
        res = minimize(self.log_likelihood, params_init, args=(data,), bounds=bounds, method=method)
        self.mu, self.a, self.b = res.x
        return res



    def predict(self, t_min, t_max, dt):
        time = np.arange(t_min, t_max, dt)
        intensity = np.zeros(len(time))
        events = []
        for i, t in enumerate(time[:-1]):
            intensity[i+1] = self.mu + np.sum(self.a * np.exp(-self.b * (time[i+1]-events)))
            if np.random.rand() < intensity[i+1] * dt:
                events.append(time[i+1])
        return time, intensity, events
    

In [101]:
from abc import ABC, abstractmethod
import matplotlib.pyplot as plt

class Process(ABC):
    @abstractmethod
    def fit(self, data):
        pass

    @abstractmethod
    def predict(self, t_min, t_max, dt):
        pass

    def performance_metrics(self, true, pred):
        mse = ((true - pred) ** 2).mean()
        return mse

    def plot_prediction(self, true, pred):
        plt.plot(true, label='True')
        plt.plot(pred, label='Predicted')
        plt.legend()
        plt.show()

class HawkesProcess(Process):
    
    
    def __init__(self, mu, a=None, b=None, c=None, intensity_func=None):
        super().__init__()
        self.mu = mu
        self.a = a
        self.b = b
        self.c = c
        self.intensity_dict = {'exp': lambda t: self.mu*np.exp(-self.a*t),
                      'power_law': lambda t: self.mu*(t+self.c) ** (-self.b)}
        
        if isinstance(intensity_func, str):
            self.intensity_func = self.intensity_dict.get(intensity_func)
        
        elif callable(intensity_func):
            self.intensity_func = intensity_func
        
        else:
            self.intensity_func = self.intensity_dict.get('exp')

    def hawkes_rhs(self, intensity, t, events):
        past_intensity = np.sum(self.intensity_func(t, events, self.a, self.b))
        return self.mu + past_intensity

    
    def log_likelihood(self, params, data):
        mu, a, b = params
        self.mu = mu
        self.a = a
        self.b = b
        events = data
        intensity = np.zeros(len(events))
        for i, t in enumerate(events[:-1]):
            intensity[i+1] = odeint(self.hawkes_rhs, intensity[i], [t, events[i+1]], args=(events[:i+1],))[-1][0]
        log_lik = -np.sum(intensity) - np.sum(np.log(intensity))
        return log_lik

    def fit(self, data, method='L-BFGS-B', bounds=None):
        from scipy.optimize import minimize
        self.events = data
        params_init = [self.mu, self.a, self.b]
        if bounds is None:
            bounds = [(1e-5, None), (1e-5, None), (1e-5, None)]
        res = minimize(self.log_likelihood, params_init, args=(data,), bounds=bounds, method=method)
        self.mu, self.a, self.b = res.x
        return res



    def predict(self, t_min, t_max, dt):
        time = np.arange(t_min, t_max, dt)
        intensity = np.zeros(len(time))
        events = []
        for i, t in enumerate(time[:-1]):
            intensity[i+1] = self.mu + np.sum(self.a * np.exp(-self.b * (time[i+1]-events)))
            if np.random.rand() < intensity[i+1] * dt:
                events.append(time[i+1])
        return time, intensity, events
    